<a href="https://www.kaggle.com/code/amaninaman/nyc-taxi-trip-records-with-gas-prices?scriptVersionId=155728701" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# MERGING NYC GAS PRICE DATA WITH NYC TAXI TRIP RECORDS DATA SET
In this notebook, I augment the analysis of NYC taxi trip records (Jan-Aug 2023) by integrating external data on weekly average motor gasoline prices from NYSERDA. After loading and exploring the main dataset, I scrape and process gas prices. The notebook then aligns trip data with gas prices based on pickup dates, addressing missing values through a fill strategy. Duplicate rows are removed, and a 'Gas Prices' column is added, representing gasoline prices in cents per gallon for each trip. The goal is to uncover correlations between taxi fares and gasoline prices, shedding light on potential patterns amid fuel cost fluctuations. Motivated by my observation that Uber prices were more affordable in Boston compared to Seattle, with a parallel difference in gas prices, I recognized the significance of considering fuel costs in such analyses. 


![](https://i.postimg.cc/Kcgh5DKp/column.png)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/nyc-taxi-trip-records-from-jan-2023-to-jun-2023/nyc_yellow_taxi_trip_records_from_Jan_to_Aug_2023.csv')
df.head(35)

Since it is going to take a lot of time to iterate 19,493,059 records, We will use random.sample() function to generate 1,000,000 unique index numbers to randomly select one million records from the dataframe. **If you are sure that your system can handle huge amount data you can skip the next three code blocks.**

In [ ]:
import random
random.seed(84)
index_list=random.sample(range(0, df.shape[0]), 1000000)
print(index_list[0:100])

In [ ]:
index_list=np.array(index_list)
print(index_list.shape)
index_list=np.sort(index_list)
index_list[0:100]

In [ ]:
df=df.iloc[index_list].sort_index()
df.reset_index(inplace=True,drop=True)
df.head(35)

The range of dates we will deal with:

In [ ]:
print(df["tpep_pickup_datetime"].max())
print(df["tpep_pickup_datetime"].min())

*to_datetime()* function in Pandas will convert string values in pickup and dropoff columns into datetime64 datatype.

In [ ]:
df["tpep_pickup_datetime"]=pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"]=pd.to_datetime(df["tpep_dropoff_datetime"])
df.info()

In this step,we will get the data from the New York State Energy Research and Development Authority (NYSERDA) website, specifically the page detailing weekly average motor gasoline prices. It uses the *requests* library to retrieve the HTML content from the specified URL. The *pd.read_html()* function from the pandas library is then employed to extract tables from the HTML content, and the resulting data is actually a set of dataframes stored in the gas_prices variable.



In [ ]:
import requests

url = requests.get('https://www.nyserda.ny.gov/Energy-Prices/Motor-Gasoline/Weekly-Average-Motor-Gasoline-Prices')
gas_prices=pd.read_html(url.text)

gas_prices


In [ ]:
NY_gas_prices=gas_prices[1]

In [ ]:
NY_gas_prices.iloc[:,0]

In [ ]:
NY_gas_prices.iloc[:,1]

We will need first and second columns containing Date and Price(Current year) data.

In [ ]:
NY_Dates_GasPrices=pd.concat([pd.to_datetime(NY_gas_prices.iloc[:,0]),NY_gas_prices.iloc[:,1]],axis="columns")
NY_Dates_GasPrices

2023-06-30 23:59:49 -
2023-01-01 00:00:52

These were the minimum and maximum values for the date column. To prepare this dataframe for matching the data on datetime values, we will filter the data to this range.

In [ ]:
NY_Dates_GasPrices.columns=["Date","Price"]
NY_Dates_GasPrices=NY_Dates_GasPrices[(NY_Dates_GasPrices['Date']>='2023-01-01') & (NY_Dates_GasPrices['Date']<='2023-07-01')]
NY_Dates_GasPrices

As it was stated, NY gas prices are showing the weekly average instead of daily prices.This code is calculating the number of days to be created first and then generating a numpy list called *date_list* by adding consecutive amount of days to the minimum date value. The idea is to create a seperate  dataframe based on days instead of weeks and then merge these records with *NY_Dates_GasPrices*.

In [ ]:
import datetime as dt

import numpy as np

date_list=np.array([])

difference=pd.Series(df["tpep_pickup_datetime"].max()-df["tpep_pickup_datetime"].min())
print(difference.dt.days[0])
print(type(difference.dt.days[0]))

for i in range(0,difference.dt.days[0]+2):
  date_item=(df["tpep_pickup_datetime"].min())+(dt.timedelta(days=i))
  date_list=np.append(date_list,date_item)


print(date_list)
print(type(date_list))
print(date_list.shape)

Let's create a Price column with NaN values. After merging *date_list* and *NY_Dates_GasPrices* dataframes, we will fill the NaN values by using *fill_na* method of Pandas library.

In [ ]:
NaN_col=pd.Series([np.NaN]*date_list.shape[0])
date_list=pd.concat([pd.Series(pd.to_datetime(date_list)),NaN_col],axis="columns")
date_list

In [ ]:
NY_Dates_GasPrices.columns=["Date","Price"]
date_list.columns=["Date","Price"]
dates_Merged=pd.concat([NY_Dates_GasPrices,date_list],axis="rows", ignore_index=True)
dates_Merged

In [ ]:
dates_Merged=dates_Merged.sort_values(by=['Date'],ignore_index=True)
dates_Merged

In [ ]:
dates_Merged['Price'].isna().value_counts()

In [ ]:
dates_Merged['Price'].fillna(method="ffill",inplace=True)
dates_Merged['Price'].fillna(method="bfill",inplace=True)
dates_Merged.head(35)

We should iterate the merged dataframe and detect the indexes of duplicating values.

In [ ]:
import datetime as dt

indexes_to_be_removed=np.array([],dtype="int32")

for i in range(0,dates_Merged.shape[0]-1):
    if dates_Merged.iloc[i,0].date()==dates_Merged.iloc[i+1,0].date():
        indexes_to_be_removed=np.append(indexes_to_be_removed,i)

    

print(indexes_to_be_removed)

In [ ]:
dates_Merged.drop(labels=indexes_to_be_removed,axis="rows",inplace=True)
dates_Merged[0:35]

In [ ]:
dates_Merged.reset_index(inplace=True, drop=True)
dates_Merged.head(35)


This code snippet iterates through the rows of df (taxi trip records) and dates_Merged to match and extract corresponding gasoline prices based on pickup dates. The nested loop efficiently populates the gas_Price_Column array with the associated prices. The use of clear_output provides a clean display, updating progress by printing the index of processed records at 1,000-record intervals. 

In [ ]:
from IPython.display import clear_output

gas_Price_Column=np.array([],dtype="float64")

for item in df.itertuples():
    for date_list_item in dates_Merged.itertuples():
        if item.tpep_pickup_datetime.date()==date_list_item.Date.date():
            matching_price=dates_Merged.loc[dates_Merged['Date'].dt.date==item.tpep_pickup_datetime.date(),'Price'].values
            gas_Price_Column = np.append(gas_Price_Column, matching_price)
    if item.Index%1000==0:
        clear_output(wait=True)
        print(str(item.Index)+ " out of "+str(df.shape[0])+' records processed.')
        print("Completed : %"+ str((item.Index /df.shape[0]) *100))

In [ ]:
df['Gas Prices']=gas_Price_Column # cents per gallon

df.info()

In [ ]:
df.plot(x="tpep_pickup_datetime",y="Gas Prices", kind="line")


In [ ]:
plt.title("Correlation between distance and tip amount")
plt.scatter(df['trip_distance'],df['tip_amount'])

plt.xlim(0.0, 150.0)
plt.xlabel("Trip Distance")

plt.ylim(0.0,200.0)
plt.ylabel("Tip Amount")
plt.show()
